In [ ]:
# Install packages if not present

#pip install coiled==0.2.13
#pip install dask==2022.03.0
#pip install itk==5.3rc4.post2
#pip install vtk
#pip install git+https://github.com/uncbiag/OAI_analysis_2.git

In [1]:
# All Imports

import coiled
import dask
from dask import compute, visualize, delayed

import time
import random
import itk
import vtk
import oai_analysis_2
from oai_analysis_2 import dask_processing as dp

In [2]:
# Start Coiled Cluster
# Enter Token when asked

import coiled
import dask
from dask import compute, visualize, delayed

cluster = coiled.Cluster(
    name='gpucluster',
    scheduler_vm_types=['t3.medium'],
    worker_vm_types=["g4dn.2xlarge", "p3.2xlarge", "p2.xlarge", "g5.2xlarge"],
    n_workers=6,
    software="pranjal-sahu/oaiimageanalysis",
    shutdown_on_close=True,
)

client = dask.distributed.Client(cluster)

Output()

Mismatched versions found

+-------------+-----------+-----------+---------+
| Package     | client    | scheduler | workers |
+-------------+-----------+-----------+---------+
| cloudpickle | 2.0.0     | 2.1.0     | None    |
| dask        | 2022.03.0 | 2022.6.0  | None    |
| distributed | 2022.3.0  | 2022.6.0  | None    |
+-------------+-----------+-----------+---------+


In [ ]:
# Main execution loop
# Change the patients_array and months_array based on requirements.
# Following code will run the computation on all samples.

atlas_path = "s3://oaisample1/ZARRDATA/atlas_image.zarr/scale0"
write_path = "/data/OAIFULLDATA/RESULTS1/"

patients_array = [
    '9283903', '9094865', '9290211', '9171580', '9214480', '9208055',
    '9000798', '9138965', '9246995', '9291234', '9068305', '9101854',
    '9316405', '9305065', '9075815', '9036316', '9155449', '9247140',
    '9128011', '9297051', '9285212', '9054866', '9007827', '9261513',
    '9263504', '9037823', '9021791', '9210505', '9239017', '9268652', 
    '9135902', '9252629', '9028786', '9277154', '9103365', 
    '9016304', '9233675', '9235073', '9212530', '9040390'
]

months_array = [
    'Month-0', 'Month-12', 'Month-24', 'Month-36', 'Month-48', 'Month-72',
    'Month-96'
]

counter = 0

print(time.time())
random.shuffle(patients_array)

for patient_id in patients_array:
    import time
    print('Processing ', patient_id, time.time())
    patient_result = []
    for month in months_array:
        image_path = "s3://oaisample1/ZARRDATA/PatientID-" + patient_id + '/' + month + '/Images/SAG_3D_DESS_0.zarr/scale0'
        
        fc_probmap, tc_probmap = dp.segment_method(image_path)
        phi_AB, image_A, image_B = dp.register_images_delayed(image_path, atlas_path)

        deformed_fc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
                                             fc_probmap, 'FC')
        deformed_tc = dp.deform_probmap_delayed(phi_AB, image_A, image_B,
                                             tc_probmap, 'TC')
        thickness_fc = dp.get_thickness(deformed_fc, 'FC')
        thickness_tc = dp.get_thickness(deformed_tc, 'TC')
        
        patient_result.append([thickness_fc, thickness_tc])
    
    output_result = compute(*patient_result)
    for result in output_result:
        itk.meshwrite(
            result[0], write_path + patient_id + '_' +
            str(counter) + '_itk_distance_inner_FC.vtk')
        itk.meshwrite(
            result[1], write_path + patient_id + '_' +
            str(counter) + '_itk_distance_inner_TC.vtk')
        counter = counter + 1
    
print(time.time())

In [ ]:
# Close the cluster after computation

cluster.close()